# **Integer Programming Method for Edge Data Distribution (EDD) problem**

In [ ]:
# Install required libraries of python-pulp.
!pip install pulp

In [ ]:
# Pick the test case to evaluate EDD.
print("Enter 1 for small input testcase OR 2 for large input test case")
n = int(input())
fileName = ""
if (n == 1):
  fileName = "/content/input.txt"
elif (n == 2):
  fileName = "/content/large_input.txt"

Enter 1 for small input testcase OR 2 for large input test case
2


In [ ]:
inputFile = open(fileName, "r")
V, E, R = [int(x) for x in inputFile.readline().split()]          # V, E, R - vertex, edges and destination edge servers.
dLimit, gamma = [int(x) for x in inputFile.readline().split()]
G = {(i, j): 0 for i in range(V+1) for j in range(V+1)}
dest_edge_servers = []
edges = []

for i in range(E):
  (u, v) = [int(x) for x in inputFile.readline().split()]
  edges.append((u, v))

for i in range(R):
  v = [int(x) for x in inputFile.readline().split()]
  dest_edge_servers.append(v[0])

inputFile.close()

Gamma = {(i, j): 1 for i in range(V+1) for j in range(V+1)}       # To keep track of the cost with any edge.
DLimit = 1 + dLimit

for (x, y) in edges:
  G[x, y] = 1
  G[y, x] = 1

for x in range(1, V+1):
  G[0, x] = 1
  Gamma[0, x] = gamma


In [ ]:
# Import pulp and create a minimize optimization problem.
import pulp as plp
opt_model = plp.LpProblem("IP_Problem", plp.LpMinimize)

In [ ]:
# H[v] to denote if edge server 'v' is visited or not in EDD.
H = {(i): plp.LpVariable(cat=plp.LpBinary, name="H_{0}".format(i)) for i in range(V+1)}

# T[u, v] to denote if edge (u, v) is used or not.
T = {(i, j): plp.LpVariable(cat=plp.LpBinary, name="T_{0}_{1}".format(i, j)) for i in range(V+1) for j in range(V+1)}

NT = {(i, j): plp.LpVariable(cat=plp.LpInteger, name="NT_{0}_{1}".format(i, j)) for i in range(V+1) for j in range(V+1)}

PT = {(i, j): plp.LpVariable(cat=plp.LpInteger, name="PT_{0}_{1}".format(i, j)) for i in range(V+1) for j in range(V+1)}

# D[v] to denote the depth of edge server 'v' in EDD IP.
D = {(i): plp.LpVariable(cat=plp.LpInteger, name="D_{0}".format(i)) for i in range(V+1)}

In [ ]:
if (DLimit == 1):           # Base condition when D_limit = 1.
  print("Optimal Solution")
  print("Optimal Cost = " + str(R*gamma))
else:
  # Optimization problem statement.
  opt_model += plp.lpSum(Gamma[i, j] * T[i, j] for i in range(V+1) for j in range(V+1))   

  # Constraint H[j] == 1 for j in destination edge servers.
  for j in dest_edge_servers:
      opt_model += H[j] == 1

  # Constraint on Depth D[j].
  opt_model += D[0] == 0
  for j in range(1, V+1):
      opt_model += D[j] <= DLimit
      opt_model += D[j] >= D[0] + 1

  # Constraint on summation of T(i, j) == H[j].
  for j in range(1, V+1):      
    opt_model += plp.lpSum([T[i, j] for i in range(V+1)]) == H[j]

  # Constraint on T[i, j] == 0, edge (i, j) is not present in G.
  for i in range(V+1):
    for j in range(V+1):
      if (G[i, j] != 1):
          opt_model += T[i, j] == 0

  # Constraint on NT to be -INF or 1 depending on T[i, j] = 0 Or 1 respectively. 
  for i in range(V+1):
    for j in range(V+1):
      opt_model += NT[i, j] == (T[i, j] - 1)*V*10 + 1

  # Constraint on PT to be +INF or 1 depending on T[i, j] = 0 Or 1 respectively.
  for i in range(V+1):
    for j in range(V+1):
      opt_model += PT[i, j] == (-1)*(T[i, j] - 1)*V*10 + 1

  # Constraint on T[c, v] >= 1, i.e., atleast one edge there must be from the cloud.
  opt_model += plp.lpSum([T[0, i] for i in range(1, V+1)]) >= 1

  # Constraint on consecutive edges depth.
  for i in range(V+1):
    for j in range(V+1):
        if(G[i, j] == 1):
          opt_model += D[j] - D[i] - NT[i, j] >= 0 
          opt_model += D[j] - D[i] - PT[i, j] <= 0

  # Constraint on T[c, v] = 1, if D[v] = 1. Obvious.
  for i in range(1, V+1):
    opt_model += T[0, i] >= (-1)*D[i] + 2

  # Constraint for having H[i] = 1, if any 'j' is visited from 'i' such that T[i, j] = 1.
  for i in range(1, V+1):
    for j in range(1, V+1):
      opt_model += T[i, j] + T[i, j] <= H[i] + H[j]

  status = opt_model.solve()

  print(plp.LpStatus[status] + " Solution")
  print("Optimal Cost = " + str(plp.value(opt_model.objective)))

  print("")

  for i in range(V+1):
      for j in range(V+1):
          if (plp.value(T[i, j]) == 1):
              print(T[i, j] , plp.value(T[i, j]))

  print("")

  for i in range(V+1):
    print(D[i] , plp.value(D[i]))


Optimal Solution
Optimal Cost = 130.0

T_0_35 1.0
T_0_36 1.0
T_0_38 1.0
T_0_48 1.0
T_0_99 1.0
T_10_18 1.0
T_13_40 1.0
T_33_34 1.0
T_35_20 1.0
T_35_32 1.0
T_35_63 1.0
T_35_92 1.0
T_35_100 1.0
T_36_13 1.0
T_36_44 1.0
T_36_49 1.0
T_36_55 1.0
T_38_42 1.0
T_38_98 1.0
T_42_6 1.0
T_42_12 1.0
T_44_2 1.0
T_44_30 1.0
T_49_22 1.0
T_50_4 1.0
T_50_28 1.0
T_55_14 1.0
T_63_16 1.0
T_92_26 1.0
T_92_46 1.0
T_98_8 1.0
T_99_10 1.0
T_99_33 1.0
T_99_50 1.0
T_100_24 1.0

D_0 0.0
D_1 2.0
D_2 3.0
D_3 2.0
D_4 3.0
D_5 2.0
D_6 3.0
D_7 2.0
D_8 3.0
D_9 2.0
D_10 2.0
D_11 2.0
D_12 3.0
D_13 2.0
D_14 3.0
D_15 2.0
D_16 3.0
D_17 3.0
D_18 3.0
D_19 2.0
D_20 2.0
D_21 2.0
D_22 3.0
D_23 2.0
D_24 3.0
D_25 2.0
D_26 3.0
D_27 2.0
D_28 3.0
D_29 2.0
D_30 3.0
D_31 2.0
D_32 2.0
D_33 2.0
D_34 3.0
D_35 1.0
D_36 1.0
D_37 2.0
D_38 1.0
D_39 2.0
D_40 3.0
D_41 2.0
D_42 2.0
D_43 2.0
D_44 2.0
D_45 2.0
D_46 3.0
D_47 2.0
D_48 1.0
D_49 2.0
D_50 2.0
D_51 2.0
D_52 3.0
D_53 2.0
D_54 2.0
D_55 2.0
D_56 2.0
D_57 2.0
D_58 3.0
D_59 2.0
D_60 2.0
D_61 2.0